Adapted from: https://www.sbert.net/examples/applications/clustering/README.html
This is a more complex example on performing clustering on large scale dataset.

This examples find in a large set of sentences local communities, i.e., groups of sentences that are highly
similar. You can freely configure the threshold what is considered as similar. A high threshold will
only find extremely similar sentences, a lower threshold will find more sentence that are less similar.

A second parameter is 'min_community_size': Only communities with at least a certain number of sentences will be returned.

The method for finding the communities is extremely fast, for clustering 50k sentences it requires only 5 seconds (plus embedding comuptation).

In this example, we download a large set of questions from Quora and then find similar questions in this set.

In [10]:
%pip install sentence_transformers

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 MB 21.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.8 MB/s eta 0:00:0000:01
  Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
  Using cached filelock-3.12.2-py3-none-any.whl (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.4/174.4 kB 16.3 MB/s eta 0:00:00
  Using cached fsspec-2023.6.0-py3-none-any.whl (163 kB)
  Using cached typing_extensions-4.7.1-py3-none-any.whl (33

  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https:/

In [3]:
from sentence_transformers import SentenceTransformer, util
import os
import csv
import time
import json

In [4]:
# Model for computing sentence embeddings. We use one trained for similar questions detection
model = SentenceTransformer('all-MiniLM-L6-v2')

In [68]:
# Get all unique sentences from the file
corpus_sentences = set()
with open('../nlp/ranada_all.json', 'r') as openfile:
    json_object = json.load(openfile)
    for line in json_object:
        text = line['text']
        if text.strip() != '':
            corpus_sentences.add(text) 

corpus_sentences = list(corpus_sentences)

In [69]:
# Encode the corpus
print("Encode the corpus. This might take a while")
corpus_embeddings = model.encode(corpus_sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

Encode the corpus. This might take a while


Batches:   0%|          | 0/580 [00:00<?, ?it/s]

In [70]:
print("Start clustering")
start_time = time.time()

#Two parameters to tune:
#min_cluster_size: Only consider cluster that have at least 40 elements
#threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters = util.community_detection(corpus_embeddings, min_community_size=40, threshold=0.78)

print("Clustering done after {:.2f} sec".format(time.time() - start_time))

#Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", corpus_sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", corpus_sentences[sentence_id])

Start clustering
Clustering done after 4.95 sec

Cluster 1, #518 Elements 
	 🇵🇭☝️💙
	 ☝☝☝🙋‍♂🙋‍♂
	 🥹🥹🥹✌️✌️✌️
	 ...
	 🤪😆😆😅
	 ❤️
	 🤣🤣🤣❤️❤️❤️

Cluster 2, #416 Elements 
	 PIA RANADA Sino ba tong tao to nga gling manira ng kapwa.dapat dyan tanggalin ng karapatan maging media
	 Pia ranada jmasyado ka matapang sino kaba. 😏 Wala kang ambag sa pilipinas panira ka lason ka sa media. Sarap mo sampalin.
	 Kulang sa pansin ang kapal ng mukha mo pia
	 ...
	 Gago tlaga magbalita yung pia ranada na yan eh..
	 HUWAG KAYONG MANIWALA SA MGA BALITA NG RAPLLER, UNA MGA SINUNGALING CLA, LALO NA C PIA RANADA, UBOD NG KAPAL NG MUKHA
	 Ang Pia sa Rappler na yan ay walang naniniwala. Salot sa lipunan.

Cluster 3, #385 Elements 
	 Mahilig mag fact check yan crappler na yan' fact check mo nga din kuya will karamihan nman sa news nila fake'
	 Hindi n nadala si ms.reporter..gu
sto po yata mawala Lalo Ang credibility Ng Rappler ..Kay president Digong Hindi kna umubra..yaan tuloy Kay taffy tulfo naka rinig k pa medyo 

In [72]:
print(len(clusters))

78


In [73]:
print(len(corpus_embeddings))

37103


In [74]:
corpus_embeddings.shape

torch.Size([37103, 384])

In [75]:
# create list of assigned cluster for each sentence
corpus_cluster = ['0']*len(corpus_sentences)

for i, cluster in enumerate(clusters):
    for sentence_id in cluster:
        corpus_cluster[sentence_id] = str(i+1)

In [19]:
import pandas as pd

In [76]:
# trying to visualize
import numpy as np
from sklearn.manifold import TSNE

X = corpus_embeddings

# reduce dimensionality
X_embedded = TSNE(n_components=2).fit_transform(X)

In [77]:
X_embedded

array([[ 2.1733910e+01, -3.1683260e+01],
       [ 2.7776896e+01, -8.1239471e+01],
       [-3.6319710e+01,  1.3663215e+01],
       ...,
       [-4.1020744e+01, -4.5177769e-03],
       [-3.5377281e+01,  2.6813475e+01],
       [ 5.6313286e+01, -3.0460917e+01]], dtype=float32)

In [78]:
# remove ones with no cluster
new_corpus_sentences = []
new_corpus_cluster = []
new_X_embedded = []

for i, cluster in enumerate(corpus_cluster):
    if cluster != '0':
        new_corpus_sentences.append(corpus_sentences[i])
        new_corpus_cluster.append(corpus_cluster[i])
        new_X_embedded.append(X_embedded[i])

In [79]:
# create a dataframe of the text and their label (cluster)
df_embeddings = pd.DataFrame(new_X_embedded)
df_embeddings = df_embeddings.rename(columns={0:'x',1:'y'})
df_embeddings = df_embeddings.assign(label=new_corpus_cluster)


In [80]:
df_embeddings = df_embeddings.assign(text=new_corpus_sentences)

In [84]:
print(len(new_corpus_sentences))

6420


In [23]:
%pip install plotly

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 28.7 MB/s eta 0:00:0000:0100:01
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils c

In [86]:
# display embedding
import plotly.express as px

fig = px.scatter(
    df_embeddings, x='x', y='y',
    color='label', labels={'color': 'label'},
    hover_data=['text'], title = 'Embedding Visualization')
fig.show()